In [4]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import datetime
import calendar
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
def collect_match_statistic_data(file_path):

    with open(file_path, mode='r') as file:
      reader = csv.reader(file)
      urls = [row[0] for row in reader]

    match_id = []
    # home_possession, away_possession = [], []
    home_total_shots, away_total_shots, home_shots_off_target, away_shots_off_target = [], [], [], []
    home_shots_saved, away_shots_saved, home_free_kicks, away_free_kicks = [], [], [], []
    home_corners, away_corners, home_fouls, away_fouls, home_offsides, away_offsides = [], [], [], [], [], []

    home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = [], [], [], []

    # chứa thông tin thẻ vàng, đỏ
    index_urls = np.array(urls).reshape(-1, 1)

    statistic_urls = []
    for idx_url in index_urls:
        statistic_url = idx_url[0].replace("index", "statistik")
        statistic_urls.append(statistic_url)

    # chứa các thông tin còn lại
    statistic_urls = np.array(statistic_urls).reshape(-1, 1)
    urls_full = np.hstack((index_urls, statistic_urls))

    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for match_i in range (urls_full.shape[0]):
        # get match id
        id = urls_full[match_i][0].split('/')[-1]
        match_id.append(id)

        statistic_response = requests.get(urls_full[match_i][1], headers=headers)

        print(urls_full[match_i][1])

        if statistic_response.status_code == 200:
            soup = BeautifulSoup(statistic_response.content, 'html.parser')

            #get home team statistic values
            home_statistic_element = soup.find_all('li', class_= 'sb-statistik-heim')
            home_statistic_values = [element.text.strip() for element in home_statistic_element]

            home_total_shots_val = int(home_statistic_values[0])
            home_shots_off_target_val = int(home_statistic_values[1])
            home_shots_saved_val = int(home_statistic_values[2])
            home_corners_val = int(home_statistic_values[3])
            home_free_kicks_val = int(home_statistic_values[4])
            home_offsides_val = int(home_statistic_values[6])
            home_fouls_val = int(home_statistic_values[5])

            home_total_shots.append(home_total_shots_val)
            home_shots_off_target.append(home_shots_off_target_val)
            home_shots_saved.append(home_shots_saved_val)
            home_corners.append(home_corners_val)
            home_free_kicks.append(home_free_kicks_val)
            home_offsides.append(home_offsides_val)
            home_fouls.append(home_fouls_val)

            #get away team statistic values
            away_statistic_element = soup.find_all('li', class_= 'sb-statistik-gast')
            away_statistic_values = [element.text.strip() for element in away_statistic_element]

            away_total_shots_val = int(away_statistic_values[0])
            away_shots_off_target_val = int(away_statistic_values[1])
            away_shots_saved_val = int(away_statistic_values[2])
            away_corners_val = int(away_statistic_values[3])
            away_free_kicks_val = int(away_statistic_values[4])
            away_offsides_val = int(away_statistic_values[6])
            away_fouls_val = int(away_statistic_values[5])

            away_total_shots.append(away_total_shots_val)
            away_shots_off_target.append(away_shots_off_target_val)
            away_shots_saved.append(away_shots_saved_val)
            away_corners.append(away_corners_val)
            away_free_kicks.append(away_free_kicks_val)
            away_offsides.append(away_offsides_val)
            away_fouls.append(away_fouls_val)

        else:
            print(f'Failed to retrieve the page. Status code: {statistic_response.status_code}')
            data = None

        index_response = requests.get(urls_full[match_i][0], headers=headers)

        print(urls_full[match_i][0])

        if index_response.status_code == 200:
            soup = BeautifulSoup(index_response.content, 'html.parser')

            home_yellow_cards_val, home_red_cards_val = 0, 0
            away_yellow_cards_val, away_red_cards_val = 0, 0
            cards_element = soup.find('div', class_='sb-ereignisse', id='sb-karten')

            if cards_element:
              # lấy cards đội nhà
              home_cards_element = cards_element.find_all('li', class_='sb-aktion-heim')
              if home_cards_element:
                home_cards = [element.text.strip() for element in home_cards_element]
                for card in home_cards:
                    if 'Yellow' in card:
                        home_yellow_cards_val += 1
                    elif 'Red' in card:
                        home_red_cards_val += 1

              # lấy cards đội khách
              away_cards_element = cards_element.find_all('li', class_='sb-aktion-gast')
              if away_cards_element:
                away_cards = [element.text.strip() for element in away_cards_element]
                for card in away_cards:
                    if 'Yellow' in card:
                        away_yellow_cards_val += 1
                    elif 'Red' in card:
                        away_red_cards_val += 1

            home_yellow_cards.append(home_yellow_cards_val)
            home_red_cards.append(home_red_cards_val)
            away_yellow_cards.append(away_yellow_cards_val)
            away_red_cards.append(away_red_cards_val)

        else:
              print(f'Failed to retrieve the page. Status code: {index_response.status_code}')
              data = None

    data = pd.DataFrame({
            'home_total_shots': home_total_shots,
            'away_total_shots': away_total_shots,
            'home_shots_off_target': home_shots_off_target,
            'away_shots_off_target': away_shots_off_target,
            'home_shots_saved': home_shots_saved,
            'away_shots_saved': away_shots_saved,
            'home_corners': home_corners,
            'away_corners': away_corners,
            'home_free_kicks': home_free_kicks,
            'away_free_kicks': away_free_kicks,
            'home_offsides': home_offsides,
            'away_offsides': away_offsides,
            'home_fouls': home_fouls,
            'away_fouls': away_fouls,
            'home_yellow_cards': home_yellow_cards,
            'away_yellow_cards': away_yellow_cards,
            'home_red_cards': home_red_cards,
            'away_red_cards': away_red_cards
    })

    data = data.dropna(subset=[
        'home_total_shots', 'away_total_shots', 'home_shots_off_target', 'away_shots_off_target',
        'home_shots_saved', 'away_shots_saved', 'home_corners', 'away_corners', 'home_free_kicks',
        'away_free_kicks', 'home_offsides', 'away_offsides', 'home_fouls', 'away_fouls',
        'home_yellow_cards', 'away_yellow_cards', 'home_red_cards', 'away_red_cards'
    ], how='all')

    return data
     